# Download images from XNAT

In [1]:
import os
path_to_repository = '/home/grg/git/xnat-monitor'
os.chdir(path_to_repository)
import bbrc

Setting up a connection to XNAT and defining which images to retrieve.

In [2]:
# Author: Grégory Operto <goperto@barcelonabeta.org>

from pyxnat import Interface
from bbrc import xnat

config_file = '/home/grg/.xnat_bsc.cfg'
dest_dir = '/tmp/downloads'
project = 'ALFA_OPCIONAL'
subjects = ['10065', '10014']
sequences = ['IR_ALFA1 SENSE']

# Setting up an active connexion to XNAT
central = Interface(config=config_file)

In [3]:
# Selecting a project from which download images
p = central.select.project(project)
# Get a list of subjects in the project to download all of them (instead of a list)
#subjects = [e.label() for e in list(p.subjects())]

for subject in subjects:
    # For each subject iterate on associated experiments
    exp = list(p.subject(str(subject)).experiments())    
    for e in exp:
        # Get a list of available sequences
        d = xnat.series_desc(central, e.id())        
        print('%s has the following sequences %s'%(subject, d))
        for sq in sequences:                    
            # Get the scan corresponding to sequence if available and select the NIFTI resource
            if sq in d.values():
                sids = [k for k,v in d.items() if v == sq and not k.startswith('0')]
                assert(len(sids)!=0)
                for sid in sids:
                    sc = e.scan(sid)
                    res = sc.resource('NIFTI')

                    # Displays a list of contained files
                    print('Resource files of sequence %s: %s'%(sq, list(res.files())))

                    # Download the corresponding archive and extract it in dest_dir
                    fp1 = res.get(dest_dir, extract=True)
                    print('%s have been created successfully'%fp1)
            else:
                print('%s is missing for subject %s'%(sq, subject))
    print('')

10065 has the following sequences {'0-OT1': 'SmartBrain SENSE', '901': 'RS_ALFA1', '601': 'B0_ALFA1', '1301': 'T2_ALFA1', '1601': 'DWI_ALFA1', '1401': 'IR_ALFA1', '1201': 'FLAIR_ALFA1', '701': 'T1_ALFA1', '404': 'Patient Aligned MPR AWPLAN_SMARTPLAN_TYPE_BRAIN', '1101': 'SE-fMRI-PA', '103': 'SmartBrain SENSE', '801': 'rRS_ALFA1', '101': 'SmartBrain SENSE', '1501': 'rDWI_ALFA1', '503': 'B1_calibration', '1001': 'SE-fMRI-AP', '501': 'B1_calibration'}
IR_ALFA1 SENSE is missing for subject 10065

10014 has the following sequences {'0-OT1': 'rRS_ALFA1 SENSE', '201': 'B1_calibration', '1201': 'rDWI_ALFA1', '202': 'B1_calibration', '601': 'RS_ALFA1 SENSE', '1301': 'DWI_ALFA1', '902': 'FLAIR_CORONAL', '301': 'T1_ALFA1', '302': 'T1_CORONAL', '303': 'T1_AXIAL', '903': 'FLAIR_AXIAL', '701': 'SE-fMRI-AP SENSE', '901': 'FLAIR_ALFA1', '401': 'B0_ALFA1', '1101': 'IR_ALFA1 SENSE', '102': 'SmartBrain SENSE', '103': 'Patient Aligned MPR AWPLAN_SMARTPLAN_TYPE_BRAIN', '801': 'SE-fMRI-PA SENSE', '101': 'Sm

Are there any missing subjects?

In [4]:
import os.path as osp
from glob import glob
import pandas as pd

wd = '/tmp/downloads/NIFTI'
t = [[subject, len(glob(osp.join(wd, '%s*'%subject)))] for subject in subjects]
df = pd.DataFrame(t, columns=['subject', 'n'])

missing = df[df['n']==0]['subject'].tolist()
print('Missing subjects: %s'%missing)

Missing subjects: ['10065']


Applying online protocol validation on first missing subject (first experiment if any)

In [5]:
first = missing[0]
expid = xnat.subject_experiments(central, project, first)[0].id()
destination = [['goperto@barcelonabeta.org'], ['goperto@barcelonabeta.org']]
res, logs = xnat.check_session(central, expid, destination=destination, send_mail=True)
print('%s resulted with %s'%(first, ['SUCCESS', 'WARNING', 'ERRORS'][res]))
logs

10065 resulted with WARNING


['Ignoring sequence 0-OT1',
 'Ignoring sequence 0-MR1',
 'Ignoring sequence 0-MR2',
 'rRS_ALFA1 SENSE is found as rRS_ALFA1',
 'RS_ALFA1 SENSE is found as RS_ALFA1',
 'SE-fMRI-AP SENSE is found as SE-fMRI-AP',
 'SE-fMRI-PA SENSE is found as SE-fMRI-PA',
 '<br> ===== Missing ========',
 'T1_CORONAL missing',
 'T1_AXIAL missing',
 'FLAIR_CORONAL missing',
 'FLAIR_AXIAL missing',
 '<br> ========= Not identified ==========',
 '<br> ========= Silently identified ==========',
 'SmartBrain SENSE silently identified',
 'SmartBrain SENSE silently identified',
 'Patient Aligned MPR AWPLAN_SMARTPLAN_TYPE_BRAIN silently identified',
 'B1_calibration silently identified',
 'B1_calibration silently identified',
 'B0_ALFA1 silently identified']